In [2]:
import math
from functools import reduce
import xml.etree.ElementTree as ET
import csv

import numpy as np
import numpy.random as random #Random samples generator
from numpy.random import normal 
from random import shuffle
import scipy
import pandas as pd

import matplotlib.pyplot as plt #Para plotar o dataset
import matplotlib.patches as mpatches
import matplotlib.mlab as mlab #Para plotar pdf da gaussiana

import time
from datetime import datetime

from IPython.display import HTML, display

primes = [ 2,   3,   5,   7,   11,  13,  17,  19,  23,  29,
           31,  37,  41,  43,  47,  53,  59,  61,  67,  71, 
           73,  79,  83,  89,  97,  101, 103, 107, 109, 113 ]

scenarios = ['C1','C2','C3','C4']

envs = { 'C1' : [[50.0,4.0],[ 92.0,18.0]],
         'C2' : [[85.0,2.0],[ 92.0,18.0]],
         'C3' : [[50.0,4.0],[155.0, 2.0]],
         'C4' : [[85.0,2.0],[155.0, 2.0]]  }

apps = ['powerline', 'battery']

files = [[0,0] for i in range(len(primes))]
dfs = [[None,None] for i in range(len(primes))]


In [3]:
# 40°C to 65°C on overhead power line;
# 65°C to 75°C on the overhead power line;
def powerline(means):
    for elem in means:
        if elem < 65:
            return True
    return False

# 40°C to 144°C on the battery.
# Over 144 °C on the battery.
def battery(means):  
    for elem in means:
        if elem > 144:
            return False
    return True

def decisor(means):
    if(powerline(means) and battery(means)):
        return 'C1'
    elif(not powerline(means) and battery(means)):
        return 'C2'
    elif(powerline(means) and not battery(means)):
        return 'C3'
    elif(not powerline(means) and not battery(means)):
        return 'C4'


In [298]:
def power_consumption(energest_value, current, voltage, time):
    # formula = (Energest_value * Current * Voltage) / (RTIMER_second * Runtime)

    return energest_value * current * voltage / (32768 * 60)

def calc_result(line):
    newline = line.replace("He:(","")
    newline = newline.replace(")","")
    means = list(map(float, newline.split(",")))
    return decisor(means)

def remove_microseconds(line):
    result = None
    line_time = line.split(".")[0]
    if len(line_time.split(":")) == 2:
        result = time.strptime(line.split(".")[0], '%M:%S')
    else:
        result = time.strptime(line.split(".")[0], '%H:%M:%S')
    result = line_time
    return result

def set_poit_to_point(dfEnergy):
    new  = [0]
    for i in range(len(dfEnergy.index)):
        if( i > 0):
            anterior = dfEnergy.index[i - 1]
            atual = dfEnergy.index[i]
            new.append((dfEnergy[atual] - dfEnergy[anterior]) * 0.33 * 3.6 / (32768 * 60) )
    return new

def select_powertrace_column(index):
    return lambda line: float(line.split(" ")[index])

def read_csv_to_pandas(filename, prime, index):
    global dfs,files
    
    df = pd.read_csv(filename,sep='\t', header=None)
    
    df = df[df[1] == 'ID:1']
    df = df.iloc[7:]
    
    df["Time"] = df[0].apply(remove_microseconds)
    df["Sensor"] = df[1]
    df["Message"] = df[2]
    
    if(index == 0):
        dfAcc = df['Message'].str.startswith('He:')
        dfEnergy = ~df['Message'].str.startswith('He:')
    else:
        dfAcc = df['Message'].str.startswith('H:')
        dfEnergy = ~df['Message'].str.startswith('H:')
    
    df["Result"] = None
    df["ALL_CPU"] = df.loc[dfEnergy,"Message"].apply(select_powertrace_column(5))
    df["ALL_LPM"] = df.loc[dfEnergy,"Message"].apply(select_powertrace_column(6))
    df["ALL_SEND"] = df.loc[dfEnergy,"Message"].apply(select_powertrace_column(7))
    df["ALL_RECEIVE"] = df.loc[dfEnergy,"Message"].apply(select_powertrace_column(8))
    
    df["CPU"] = None
    df["LPM"] = None
    df["SEND"] = None
    df["RECEIVE"] = None
    
    df["CPU"].loc[dfEnergy] = set_poit_to_point(df['ALL_CPU'].loc[dfEnergy])
    df["LPM"].loc[dfEnergy] = set_poit_to_point(df['ALL_LPM'].loc[dfEnergy])
    df["SEND"].loc[dfEnergy] = set_poit_to_point(df['ALL_SEND'].loc[dfEnergy])
    df["RECEIVE"].loc[dfEnergy] = set_poit_to_point(df['ALL_RECEIVE'].loc[dfEnergy])
        
    df = df.filter(["Time","Message", "Result", "Energy","ALL_CPU","ALL_LPM","ALL_SEND","ALL_RECEIVE","CPU","LPM","SEND","RECEIVE"])
    df = df.iloc[0:121]    
    files[prime][index] = 1
    dfs[prime][index] = df 

In [355]:
for j in range(len(primes)):
    i = primes[j]
    
    print("Prime: %d" % (i))
    filename = "../../tests/hercules/output-c1-"+str(i)+".tsv" 
    read_csv_to_pandas(filename, j, 1)

    filename = "../../tests/hepheastus/output-c1-"+str(i)+".tsv"
    read_csv_to_pandas(filename, j, 0)

flag = 0
for i in range(1):
    if( files[i][0] != 1):
        flag = 1
        print("Falta Coletar Exemplo do hepheastus com semente: %d" % i)
    if( files[i][1] != 1):
        flag = 1
        print("Falta Coletar Exemplo do hercules com semente: %d" % i)
        
if(flag == 0):
    print("Tudo ok com as coletas")

df = dfs[0][0]

Prime: 2
Prime: 3
Prime: 5
Prime: 7
Prime: 11
Prime: 13
Prime: 17
Prime: 19
Prime: 23
Prime: 29
Prime: 31
Prime: 37
Prime: 41
Prime: 43
Prime: 47
Prime: 53
Prime: 59
Prime: 61
Prime: 67
Prime: 71
Prime: 73
Prime: 79
Prime: 83
Prime: 89
Prime: 97
Prime: 101
Prime: 103
Prime: 107
Prime: 109
Prime: 113
Tudo ok com as coletas


In [380]:
def print_values(df):
    dfAcc = df['Message'].str.startswith('He:')
    dfEnergy = ~df['Message'].str.startswith('He:')

    df.loc[dfAcc, 'Result'] = df.loc[dfAcc,"Message"].apply(calc_result)
    
    df["CPU_SEND"] = None
    df["CPU_SEND"].loc[dfEnergy] = df.loc[dfEnergy, ["CPU","SEND"]].sum(axis=1)
    
#     print(df["CPU"] > df["CPU_SEND"])

    return { "cpu": df["CPU"].sum(), "cpu_send": df["CPU_SEND"].sum() }

stats = {}
for j in range(len(primes)):
    i = primes[j]
    stats[j] = {}
    print("Prime: %d" % j)
#     print("Hepheastus: ")
    stats[j][0] = print_values(dfs[j][0])
#     print("Hercules: ")
    stats[j][1] = print_values(dfs[j][1])

Prime: 0
Prime: 1
Prime: 2
Prime: 3
Prime: 4
Prime: 5
Prime: 6
Prime: 7
Prime: 8
Prime: 9
Prime: 10
Prime: 11
Prime: 12
Prime: 13
Prime: 14
Prime: 15
Prime: 16
Prime: 17
Prime: 18
Prime: 19
Prime: 20
Prime: 21
Prime: 22
Prime: 23
Prime: 24
Prime: 25
Prime: 26
Prime: 27
Prime: 28
Prime: 29


In [405]:
print("Hepheastus: ")
cpu_hepheastus = 0
cpu_send_hepheastus = 0
cpu_var_hepheastus = 0
cpu_send_var_hepheastus = 0

cpu_hercules = 0
cpu_send_hercules = 0
cpu_var_hercules = 0
cpu_send_var_hercules = 0

for j in range(len(primes)):
    cpu_hepheastus += stats[j][0]["cpu"]
    cpu_send_hepheastus += stats[j][0]["cpu_send"]
    
    
    cpu_hercules += stats[j][1]["cpu"]
    cpu_send_hercules += stats[j][1]["cpu_send"]
    
for j in range(len(primes)):
    cpu_var_hepheastus += (stats[j][0]["cpu"] - cpu_hepheastus/30)**2
    cpu_send_var_hepheastus += (stats[j][0]["cpu_send"] - cpu_send_hepheastus/30)**2
    
    cpu_var_hercules += (stats[j][1]["cpu"] - cpu_hercules/30)**2
    cpu_send_var_hercules += (stats[j][1]["cpu"] - cpu_send_hercules/30)**2
    
    
print("Hepheastus: ")
print("Mean CPU: %f" % (cpu_hepheastus/30))
print("Mean CPU_SEND: %f" % (cpu_send_hepheastus/30))
print("Sd: %f\n\n" % math.sqrt(cpu_send_var_hepheastus/29))

print("Hercules: ")
print("Mean CPU: %f" % (cpu_hercules/30))
print("Mean CPU_SEND: %f" % (cpu_send_hercules/30))
print("Sd: %f\n\n\n\n" % math.sqrt(cpu_send_var_hercules/29))

print("Quantidade de minutos")
print("Hepheastus: ")
print("Mean CPU: %.2f minutes" % (60*7.8/(cpu_hepheastus/30)))
print("Mean CPU_SEND: %.2f minutes\n\n" % (60*7.8/(cpu_send_hepheastus/30)))

print("Hercules: ")
print("Mean CPU: %.2f minutes" % (60*7.8/(cpu_hercules/30)))
print("Mean CPU_SEND: %.2f minutes" % (60*7.8/(cpu_send_hercules/30)))


Hepheastus: 
Hepheastus: 
Mean CPU: 1.470500
Mean CPU_SEND: 1.627234
Sd: 0.056356


Hercules: 
Mean CPU: 1.206496
Mean CPU_SEND: 1.358005
Sd: 0.159504




Quantidade de minutos
Hepheastus: 
Mean CPU: 318.26 minutes
Mean CPU_SEND: 287.60 minutes


Hercules: 
Mean CPU: 387.90 minutes
Mean CPU_SEND: 344.62 minutes
